In [139]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import torch
import gpytorch

### Seasonality

Data is collected at 5 minute intervals over a 10 day timespan

Frequency = Number of observations per _season_ or _cycle_

Season = Hour, Day, Week, Month, Quarter, Year

_Note_: In Fourier analysis, “period” is the length of the cycle, and “frequency” is the inverse of period

Daily seasonality frequency of $288 = (24 * 60) / 5$

Day of week seasonality frequency of $2016 = (288 * 7)$

Work Week seasonality frequency of $1440 = (288 * 5)$

In [140]:
#sigma_line = query_table(table='uv_sigma_line_eg_5T')
sigma_line = pd.read_csv('/Users/wastechs/Downloads/machine_day12.csv')

In [141]:
sigma_line.set_index(sigma_line.t, inplace=True)
del sigma_line['t']

In [ ]:
px.line(data_frame=sigma_line, x=sigma_line.index, y='p')

### Scale $X$ between [0-1] and use double precision

In [142]:
sigma_line.shape

(1728, 9)

In [137]:
#sigma_line = sigma_line[sigma_line.index.day >= 12] ## for pedagoical reasons

time_int_range = np.arange(0, 1728*5, 5)
sigma_line['t'] = time_int_range

X = (sigma_line['t'].values - np.min(sigma_line['t'].values)) / (np.max(sigma_line['t'].values) - np.min(sigma_line['t'].values))
y = sigma_line['kw'].values

n = len(X)

prop_train = 0.7
n_train = round(prop_train * n)

X_train = torch.from_numpy(X[:n_train]).to(torch.double)
y_train = torch.from_numpy(y[:n_train]).to(torch.double)

y_train_mean = torch.mean(y_train)
y_train_std = torch.std(y_train)

X_test = torch.from_numpy(X[n_train:]).to(torch.double)
y_test = torch.from_numpy(y[n_train:]).to(torch.double)

### Keep $X$ the same and use float32

In [138]:
time_int_range = np.arange(0, 1728*5, 5)
all['t'] = time_int_range

X = all['t'].values
y = all['kw'].values

n = len(X)

prop_train = 0.7
n_train = round(prop_train * n)

X_train = torch.from_numpy(X[:n_train]).to(torch.float32)
y_train = torch.from_numpy(y[:n_train]).to(torch.float32)

y_train_mean = torch.mean(y_train)
y_train_std = torch.std(y_train)

X_test = torch.from_numpy(X[n_train:]).to(torch.float32)
y_test = torch.from_numpy(y[n_train:]).to(torch.float32)

In [ ]:
class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)

        self.mean_module = gpytorch.means.ConstantMean()
        period_prior = gpytorch.priors.NormalPrior(30, 1.5)
        period_constraint = gpytorch.constraints.Interval(10, 60)
        outputscale_prior = gpytorch.priors.GammaPrior(0.5, 0.15)
        
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.PeriodicKernel(
                period_length_prior=period_prior,
                period_length_constraint=period_constraint
                ) * gpytorch.kernels.RBFKernel(),
                outputscale_prior=outputscale_prior
            )

        # Initialize lengthscale and output scale to mean of priors
        #self.covar_module.outputscale = outputscale_prior.mean
        #self.covar_module.base_kernel.lengthscale = lengthscale_prior

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X_train, y_train, likelihood)

Actual learned parameters of the model are things like raw_noise, raw_outputscale, raw_lengthscale, etc. The reason for this is that these parameters must be positive

In [ ]:
print('Actual outputscale:', model.covar_module.outputscale)

In [ ]:
model.state_dict()

In [ ]:
model.double()
likelihood.double()

model.train()
likelihood.train()

In [ ]:
with gpytorch.settings.max_cg_iterations(5000):
    # Use the adam optimizer
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},  # Includes Gaussian Likelihood parameters
        ], lr=0.01)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iter = 300
    for i in range(training_iter):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        
        # Output from model
        output = model(X_train)

        # Calc loss and backprop gradients
        loss = -mll(output, y_train)
        loss.backward()

        if (i % 20) == 0:
            print('Iter %d/%d - Loss: %.3f noise: %.6f' % (
                i + 1, training_iter, loss.item(),
                model.likelihood.noise.item()
            ))
            print(list(model.parameters()))
        optimizer.step()

In [ ]:
raw_outputscale = model.covar_module.raw_outputscale
constraint = model.covar_module.raw_outputscale_constraint

print('Transformed outputscale: ', constraint.transform(raw_outputscale))

In [ ]:
print('Actual outputscale:', model.covar_module.outputscale)

In [ ]:
# Get into evaluation (predictive posterior) mode
model.eval()
likelihood.eval()

# Make predictions by feeding model through likelihood
with torch.no_grad(), gpytorch.settings.fast_pred_var(), gpytorch.settings.max_cg_iterations(9000):
    #test_x = test_x.to(torch.float32)
    test_pred = likelihood(model(X_test))
    train_pred = likelihood(model(X_train))

In [ ]:
with torch.no_grad():

    # Initialize plot
    f, ax = plt.subplots(1, 1, figsize=(20, 6))
    # Get upper and lower confidence bounds
    #lower, upper = observed_pred.confidence_region()
    # Plot training data as black stars
    ax.plot(X_train.numpy(), y_train.numpy(), alpha=0.7)
    ax.plot(X_train.numpy(), train_pred.mean.numpy())
    # Plot predictive means as blue line
    ax.plot(X_test.numpy(), y_test.numpy())
    ax.plot(X_test.numpy(), test_pred.mean.numpy(), lw=4)
    # Shade between the lower and upper confidence bounds
    #ax.fill_between(X_test.numpy(), lower.numpy(), upper.numpy(), alpha=0.5, color='#2ecc71')
    ax.legend(['Observed Data', 'Training', 'Test', 'Predictions'])